# Tutorial 1: From model to observations

## Solar System as HD163296

**Download this tutorial contents from** [here](https://keeper.mpdl.mpg.de/d/7fe6524c7ec945168630/)**, including the project, models and script.**

One key question to connect the origin of our Solar System to the general understanding of the planet-formation field is to estimate **how would the Solar System have looked like if ALMA had observed it**. In [Bergez-Casalou et al. 2022](https://ui.adsabs.harvard.edu/abs/2022A%26A...659A...6B/abstract), hydro-models were complemented with dust evolution models to answer this question.

By running radiative transfer codes with the dust distribution of the hypothetical Solar System, it is possible to obtain a prediction of the brightness of the Solar System planet-forming disk at 1.3mm wavelengths. The following figure shows such prediction, with each line showing the approximate orbit position of Jupiter, Saturn, Uranus, and Neptune, from the inside out, respectively.

<img src="./SolarS_original.png">
<center> Figure 1: The 1.3mm continuum emission from the young Solar System as simulated by Bergez-Casalou et al. 2022. </center>

Radiative transfer images are **the** fundamental input for *SIMIO-continuum*. The package will take this image and generate a synthetic observation based on its brightness distribution. The details of how this is done is covered in "[How does SIMIO generates the observations?](https://simio-continuum.readthedocs.io/en/latest/content/content_3.html)" In the following steps, we will cover in detail the procedure to go from a model image to an ALMA observation.

## Step 1: The package

[Download](https://www.nicolaskurtovic.com/simio/download-simio) the *SIMIO-continuum* package (from now on, referred to as *SIMIO*), and open the *SIMIO* folder. You will see the following:

- **codes**: Where the functions and wrappers are located.

- **plots**: After creating your synthetic observation, you can use the functions within this folder to generate figures and radial profiles. Check these codes if you do not have experience with fits files.

- **projects**: Store your projects in this folder.

- **templates**: Store your templates in this folder.

- **casa_examples**: Example codes to run *SIMIO* are stored in this folder.

- **simio_casa.py**: Example code to use *SIMIO*. Any *SIMIO* code should be run from this location (*~/path_to_simio/simio/*).

<img src="./step1_web.png">

## Step 2: Include your template

We want to know how the Solar System would have looked like if it had been observed as HD163296 from DSHARP. Go into the **templates/** folder and add the HD163296 template. You can download all the publicly available templates from this [page](https://www.nicolaskurtovic.com/simio/templates).

Each template is an archival ALMA observation adapted to run with *SIMIO*. By selecting a template, you will match the same uv-coverage, exposure time, position of the object in the sky, frequency bandwidth, and all the technical parameters of such observation.

<img src="./step2_web.png">

## Step 3: Create your project

Create a folder with the name of your project in the folder **projects/**. You can use any name you want for this folder. In this example, the name of our project will be "*SolarS_HD163296*" since we want to generate an observation of how the Solar System would look like if DSHARP had observed it at the distance and geometry of HD163296.

<img src="./step3_web.png">

## Step 4: Prepare your project

Go inside your project folder. In this example, we are inside the folder **SolarS_HD163296/**. Create the folders "*images*", "*msfiles*", "*uvtables*", and leave them empty. Add your radiative transfer image either in "*.out*" format (standard output format from [RADMC3D](https://www.ita.uni-heidelberg.de/~dullemond/software/radmc-3d/)), or "*.npy*" format. In this example, our radiative transfer image is "*image_1300micron.out*", shown at the beginning of this tutorial. Alternatively, a file can be saved to "*.npy*" format by saving a *NumPy array* with the function *np.save(array)*. Therefore, any image stored as a *NumPy matrix* can also be input in SIMIO.

**Note:** The input image must be as big as the field of view, and the pixel size must be at least five times smaller than the highest angular resolution. Bigger images and smaller pixel sizes will produce a more stable Fourier Transform of the models by reducing the short baselines artifacts and smoothing the brightness difference from pixel to pixel.

*SIMIO* will use the folders you just created to store:

- **images**: The fits files generated with *SIMIO* will be stored in this folder. Check it after running the code in Step 5.

- **msfiles**: Your synthetic observation will have its own measurement file, where the visibilities are stored. You will be able to find that file in this folder.

- **uvtables**: The visibilities will also be given in ".txt" format, which you can further use to analyze with alternative tools, such as [frank](https://discsim.github.io/frank/index.html) or [galario](https://mtazzari.github.io/galario/index.html). *SIMIO* will write the visibility table inside this folder if you calculate the visibilities with the *get_mod_ms* function.

<img src="./step4_web.png">

## Step 5: Run SIMIO

Go back to the initial *SIMIO* folder. Open simio_casa.py and open CASA 5.6.2 in a terminal. **Note:** *SIMIO* should work on any CASA 5.X version, but 5.4 or 5.6 should be preferred, as most of the testing has been done in those. You can find the links to download the CASA software [here](https://casa.nrao.edu/casa_obtaining.shtml).

The first part of the *simio_casa.py* code will import all the necessary python packages in CASA and set the path to the *SIMIO* folder.

```python
# Import needed python packages
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Get the current directory path
current_dir = os.getcwd()+'/'
```

The second part of the code is needed to load the *SIMIO* functions. Each subcode needs functions from the previous, and so it is necessary to execute them in the correct order. The [analysis utils](https://casaguides.nrao.edu/index.php/Analysis_Utilities) of CASA are included in the *SIMIO* package for self-containing purposes.

You should not need to change anything in the import and execfile blocks.

```python
# Import the analysis utils functions
sys.path.append(current_dir+'codes/analysis_scripts/')
import analysisUtils as au


# Import the simio object
execfile(current_dir+'codes/simio_obj.py')
# Import functions for uv-handling
execfile(current_dir+'codes/simio_ms2ascii.py')
# Import functions for imaging
execfile(current_dir+'codes/simio_clean.py')
```

Create your *simio_object*, the primary object of the *SIMIO* package. This object will contain all the needed information to generate your synthetic observation.

- **object_name**: Write the name of your project, which you created in steps 3 and 4.

- **im_file_name**: Name of your radiative transfer image, including the format (.out or .npy). **Note:** If your image file is .npy, you will need to specify the pixel size. Check the other tutorials for an example.

- **template**: Name of the template you want to mimic. In this example, HD163296.

- **use_tempgeom**: Set to True if you want *SIMIO* to incline and rotate your image with the inclination and position angle of the template, under the assumption that your input is face-on. If your image already has the geometry you desire, and you do not want *SIMIO* to apply any geometric change, then set it to False. **Note** You can also use just a rotation or incline. Please check the other tutorials.

```python
###########################
# Solar System as HD163296
###########################

# Create a simio object.
simobj = simio_object(object_name  = 'SolarS_HD163296', 
                      im_file_name = 'image_1300micron.out', 
                      template     = 'HD163296', 
                      use_tempgeom = True)
```

The function *get_mod_ms_ft* will take your image and generate the *measurement set* (which is the object that contains all the information of an ALMA observation) as if it had been observed with the same technical setup of your template. Running this function will give you the path to the generated *measurement set* in the variable *mod_ms*. The time this function takes to run will depend on the computer and template. As a reference, most templates take ~4min on an average laptop.

This line is where the Fourier Transform of your model is computed. For each spectral window of the real observation, *SIMIO* will replace the DATA COLUMN with the visibilities of your model, therefore matching exactly the uv-coverage. In other words, you will have the same angular resolution.

The function *get_mod_ms_ft* will use the Fourier Transform from CASA, while *get_mod_ms* will use the same as *galario*. If you use the latter, the visibility table will be written in ".txt" as part of the process.

```python
# Create the measurement file of your simio object
# Can take several minutes
mod_ms = get_mod_ms_ft(simobj)
```

Now that the *measurement set* has been generated, the following step is to create the images of your synthetic observation. We will do this with the CLEAN algorithm.

Create the masks to [CLEAN](https://casa.nrao.edu/docs/taskref/tclean-task.html) the synthetic observation. These masks will be loaded into the *simobj*and also returned as a string. They will have the standard of [CASA Regions](https://casaguides.nrao.edu/index.php/CASA_Region_Format).

- **mask_obj**: An elliptical mask with the geometry of the template. You can set the semi-major axis of the ellipse in units of arcsec. 

- **mask_res**: Generates an annulus mask needed to calculate the background properties of your image.

```python
# Create Masks
mask_obj = simobj.get_mask(mask_semimajor=0.65) # in arcsec
mask_res = simobj.get_residual_mask()
```

Now we need to generate the images of your synthetic observation. The function *easy_mod_tclean* is a wrapper of the function *tclean* for *simobj*. It runs the CLEAN algorithm over your generated observation.

Set *interactive* to *True* to check if the mask includes all the emission. Press the green arrow (in the Next Action section) to start a cycle of the cleaning process, or press the blue arrow and wait until it is done. **Note:** Use the blue arrow only if you have tested convergence of the current threshold.

<img src="./tut1_panel_a.png">

The image looks structured and noisy, as this is the "dirty image" resulting from convolving the input image with the dirty beam. Please check the imaging tutorial of CASA for more information. Depending on your computer and the template, cleaning the image can take a few minutes to a fraction of an hour. Be patient.

## Step 6: Check the results

After finishing the cleaning process, go back to the project folder to check the *SIMIO* products. The execution of *get_mod_ms_ft* generated a *measurement set* in the *msfiles* folder (the one you created in step 4). After executing *easy_mod_tclean* at the end of Step 5, you will get the products in the images folder.

The images will be named by your project name, plus a suffix. Each image is:

- **project_im.fits**: Beam convolved image, with the JvM correction ([Czekala et al. 2021](https://ui.adsabs.harvard.edu/abs/2021ApJS..257....2C/abstract)). This image is how your source would look if ALMA had observed it with the same observational setup as the template.

- **project_im_model.fits**: The model image, generated by the CLEAN algorithm as a description of the visibilities of your source in the sky plane.

- **project_im_noJvM.fits**: Beam convolved image, without the JvM correction.

- **project_im_psf.fits**: The PSF of the observation.

- **project_im_residual.fits**: The residuals of the CLEAN algorithm. This residual image will be very structured, as the generated ms file does not contain noise. However, it should be negligible in total flux.

<img src="./step6_web.png">

The standard imaging product of ALMA is the beam convolved image, which is called "project_im.fits" or "SolarS_HD163296_im.fits" in this tutorial. You can visualize that image with the task casaviewer from CASA or the software [DS9](https://sites.google.com/cfa.harvard.edu/saoimageds9). You can also open those files in python with the package [astropy](https://www.astropy.org/) (Check the plots folder for examples).

If the Solar System was 1Myr old, located at the distance of HD163296, and had been observed by the DSHARP survey, then it would look like the following figure, where it is compared to the template disk.

<img src="./SolarS_HD163296_comparison.png">

That's all! :D

Now you have a synthetic observation of the Solar System, if it was located at the same distance of HD163296, at the same position in the sky, and if DSHARP had observed it. You can see that the Solar System would have looked like a compact disk with a cavity when observed through the eyes of ALMA.

You can take all the products and study them however you want: In the visibility plane or the image plane. You can also use the measurement set to generate images with different angular resolutions. Check the other tutorials to see what else you can do with *SIMIO*.